In [ ]:
import sys
import os
from icecream import ic

from pathlib import Path

import utils_behavior

from utils_behavior import Ballpushing_utils
from utils_behavior import Utils
from utils_behavior import Processing
from utils_behavior import HoloviewsTemplates

import pandas as pd
import hvplot.pandas
import numpy as np


# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
import pandas as pd
#import pyarrow.feather as feather

import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.pandas



import matplotlib.pyplot as plt
import seaborn as sns



import holoviews as hv


hv.extension("bokeh")

In [ ]:
# Get the path to the data

Datapath = Utils.get_data_path()

In [ ]:
# Find folders with "Learning or learning" in the name as a list

folders = [f for f in Datapath.glob("*FeedingState*")]

folders

In [ ]:
# Take the first 

exp = Ballpushing_utils.Experiment(folders[0])

In [ ]:
exp

In [ ]:
# For each fly, get the ball positions

ball_positions = []

for fly in exp.flies:
    data = fly.balltrack.objects[0].dataset
    
    data["y_relative"] = abs(data["y_centre"] - data["y_centre"].iloc[0])
    data['fly'] = fly.name
    
    ball_positions.append(data)
    
ball_positions = pd.concat(ball_positions)

ball_positions.head()

#ball_positions['fly'].unique()
    
    

In [ ]:
ball_positions[]

In [ ]:
# Now get the average ball positions grouped by time

grouped = ball_positions.groupby('time')["y_relative"].mean()

# Make it a dataframe

grouped = grouped.reset_index()

# Add co
grouped["ci"] = (ball_positions.groupby('time')["y_relative"].sem()) * 1.96


grouped

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress

sns.set_style("white")
sns.set_style("ticks")

# For each fly, get the ball positions
ball_positions = []

for fly in exp.flies:
    data = fly.balltrack.objects[0].dataset
    
    data["y_relative"] = abs(data["y_centre"] - data["y_centre"].iloc[0])
    data['fly'] = fly.name
    
    ball_positions.append(data)
    
ball_positions = pd.concat(ball_positions)

# Sample 20 flies from the pool of exp.flies
sampled_flies = np.random.choice(ball_positions['fly'].unique(), 20, replace=False)

# Now get the average ball positions grouped by time
grouped = ball_positions.groupby('time')["y_relative"].mean().reset_index()

# Plot the individual ball relative positions over time in thin gray lines for the sampled flies
plt.figure(figsize=(10, 6))
for fly in sampled_flies:
    sns.lineplot(x='time', y='y_relative', data=ball_positions[ball_positions['fly'] == fly], color='gray', alpha=0.3, linewidth=0.5)

# Plot the average ball positions over time in a thicker line
sns.lineplot(x='time', y='y_relative', data=grouped, label='Average ball position', color='blue', linewidth=2)

# Add a linear regression line to the plot and annotate the R2
slope, intercept, r_value, p_value, std_err = linregress(grouped['time'], grouped['y_relative'])
plt.plot(grouped['time'], slope * grouped['time'] + intercept, color='red', linestyle='--', alpha=0.5, label='Linear regression', linewidth=1)

# Add the R2 to the plot
plt.text(6500, 110, f"R2 = {r_value**2:.2f}", fontsize=12, color='red')

# Add title and labels
plt.title('Average ball position over time')
plt.xlabel('Time (s)')
plt.ylabel('Position relative to starting position (px)')
#plt.legend()
plt.savefig("/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/Article_Affordance/average_ball_position_over_time.png")
plt.savefig("/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/Article_Affordance/average_ball_position_over_time.svg")
# Show the plot
plt.show()

In [ ]:
Dataset = Ballpushing_utils.Dataset(exp)

In [ ]:
Dataset.generate_dataset(success_cutoff=False)

In [ ]:
Dataset.data.columns

In [ ]:
# Plot the yball_relative as a function of time grouped by individual flies

# curves = hv.Curve(Dataset.data, kdims=["time"], vdims=["yball_relative","fly"]).groupby("fly").overlay()

# curves.opts(width=800, height=400)

In [ ]:
from holoviews import opts
from bokeh.palettes import viridis

Tries = 30

NbFlies = 7

for i in range(Tries):

    # Pick random flies and plot their yball_relative as a function of time
    flies = np.random.choice(Dataset.data.fly.unique(), NbFlies)

    # Generate a Viridis palette with adapted number of colors colors
    viridis_palette = viridis(NbFlies)

    # Create the curves and overlay them
    curves = (
        hv.Curve(
            Dataset.data[Dataset.data.fly.isin(flies)],
            kdims=["time"],
            vdims=["yball_relative", "fly"],
        )
        .groupby("fly")
        .overlay()
    )

    curves = curves.opts(
        opts.Curve(
            width=800,
            height=400,
            show_grid=True,
            show_legend=False,
            ylabel="ball position relative to start (px)",
            xlabel="time (s)",
            
        )
    )

    # Apply the viridis colormap
    curves = curves.opts(opts.Curve(color=hv.Cycle(values=list(viridis_palette))))

    hv.save(curves, f"/mnt/upramdya_files/DURRIEU_Matthias/DlsImgs/testcurves{i}.png")